# Single Responsibility Principle

"A class should have one, and only one, responsibility and reason to change"

This principle states that a class should have one and only one responsibility and reason to change.

* Database code in Video class (Violates Single Responsibility Principle)

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

DATABASE_URL = "mysql+mysqlconnector://root:@localhost/youtube"

engine = create_engine(DATABASE_URL)
Base = declarative_base()

class Video(Base):
    __tablename__ = "Video"

    id = Column(Integer, primary_key=True, autoincrement=True)
    title = Column(String(255))
    time = Column(Integer)
    likes = Column(Integer)
    views = Column(Integer)

    def __init__(self, title, time, likes, views):
        self.title = title
        self.time = time
        self.likes = likes
        self.views = views
        self.engine = create_engine(DATABASE_URL)
        Base.metadata.create_all(self.engine)
        Session = sessionmaker(bind=self.engine)
        self.session = Session()

    def get_number_of_hours_played(self):
        return (self.time / 3600.0) * self.views

    def persist(self):
        try:
            self.session.add(self)
            self.session.commit()
            print("Video persisted successfully.")
        except Exception as e:
            self.session.rollback()
            print(f"Error persisting video: {e}")
        finally:
            self.session.close()

* Applying Single Responsibility Principle

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

DATABASE_URL = "mysql+mysqlconnector://root:@localhost/youtube"

engine = create_engine(DATABASE_URL)
Base = declarative_base()

class Video(Base):
    __tablename__ = "Video"

    id = Column(Integer, primary_key=True, autoincrement=True)
    title = Column(String(255))
    time = Column(Integer)
    likes = Column(Integer)
    views = Column(Integer)

    def __init__(self, title, time, likes, views):
        self.title = title
        self.time = time
        self.likes = likes
        self.views = views

    def get_number_of_hours_played(self):
        return (self.time / 3600.0) * self.views

class DatabaseConnection:
    def __init__(self, db_url):
        self.engine = create_engine(db_url)
        Base.metadata.create_all(self.engine)
        Session = sessionmaker(bind=self.engine)
        self.session = Session()

    def get_session(self):
        return self.session

    def close_session(self):
        self.session.close()


class VideoDAO:
    def __init__(self, db_connection):
        self.db_connection = db_connection
        self.session = db_connection.get_session()

    def persist(self, video):
        try:
            self.session.add(video)
            self.session.commit()
            print("Video persisted successfully.")
        except Exception as e:
            self.session.rollback()
            print(f"Error persisting video: {e}")
        finally:
            self.db_connection.close_session()

### Single Responsibility Principle (SRP) Explanation

The Single Responsibility Principle (SRP) states that a class should have only one reason to change, meaning it should have only one job or responsibility.

**First Code (Violation of SRP):**

The initial `Video` class had two primary responsibilities:

1.  **Data Representation:** Holding video attributes like `title`, `time`, `likes`, and `views`.
2.  **Database Interaction:** Managing database connection, creating tables, and persisting video data.

**Problem:** The class also handles database creation, which should be done by separate class or class only dealing with database stuff.

**Why is this bad?**

*   If the database connection details (like the URL) need to change, or if the persistence logic needs to be updated (e.g., switching to a different ORM), you'd have to modify the `Video` class. This change might unintentionally affect the video data representation logic.
*   Testing becomes more complex because you can't easily test the video data representation logic in isolation from the database interaction logic.

**Second Code (Adherence to SRP):**

The second code demonstrates better adherence to the SRP by separating responsibilities into distinct classes:

*   **`Video` Class:** *Solely* responsible for representing video data and calculating `get_number_of_hours_played`. It doesn't deal with database interactions.
*   **`DatabaseConnection` Class:** Manages the database connection, engine creation, and session management.
*   **`VideoDAO` Class:** Handles the data access operations (persistence) for the `Video` class.

**Solution:**
* Separate class for each responsibility.

**Why is this better?**

*   Each class has a single, well-defined purpose.
*   Changes to the database connection or persistence logic only affect the `DatabaseConnection` or `VideoDAO` classes, without impacting the `Video` class.
*   Testing becomes easier because each class can be tested independently.
*   Makes code more readable, easier to maintain and modify.